In [ ]:
#seleziono x posts da un profilo
from instaloader import Instaloader, Profile

L = Instaloader()

profile = Profile.from_username(L.context, 'username_profile')

count=1
for post in profile.get_posts():
    L.download_post(post, target=profile.username)
    count +=1
    if count ==16:
        break

In [ ]:
#scaricare post da una certa data ad una certa data

from instaloader import Instaloader, Profile

L= Instaloader()

from datetime import datetime
from itertools import dropwhile, takewhile


#posts = L.get_hashtag_posts('hashtag')
# or

PROFILE='username_profile'
posts = instaloader.Profile.from_username(L.context, PROFILE).get_posts()

SINCE = datetime(2019, 7, 14)
UNTIL = datetime(2019, 1, 1)

for post in takewhile(lambda p: p.date > UNTIL, dropwhile(lambda p: p.date > SINCE, posts)):
    print(post.date)
    L.download_post(post, target=Profile.username)

In [1]:
import json
import glob
l = glob.glob(r"C:\Users\valen\instagram_posts\*UTC.json")
detta = []
for nome in l:
    with open(nome, 'r') as file:
        data = file.read().replace('\n', '')
    d = json.loads(data)
    detta.append(d)

In [2]:
import pandas as pd
detts = pd.DataFrame(detta)

In [3]:
len(detts)

95

In [ ]:
import pprint
pprint.pprint(detta)

In [6]:
from datetime import datetime
import re

In [7]:
newl=[]
for d in detta:
    edge_media_to_caption = d['node']['edge_media_to_caption']['edges']
    for e in edge_media_to_caption:
        caption = e['node']['text']
        for has in caption:
            hashtag1=re.findall(r'\#\w+', caption)
    id_post = d['node'] ['id']
    nr_comments = 0
    edge_media_preview_comment = 0
    if('edge_media_preview_comment' in d['node']):
        nr_comments = d['node'] ['edge_media_preview_comment'] ['count']
        edge_media_preview_comment = d['node'] ['edge_media_preview_comment'] ['edges']
    nr_likes = d['node'] ['edge_media_preview_like'] ['count']
    location_name = ""
    lat = 0
    lng = 0
    if(('location' in d['node']) and ( d['node']['location'] != None)):
        if('name' in d['node']['location']):
            location_name = d['node']['location']['name']
            lat = d['node'] ['location'] ['lat']
            lng = d['node'] ['location'] ['lng']
    followers = d['node'] ['owner'] ['edge_followed_by'] ['count']
    followed = d['node'] ['owner'] ['edge_follow'] ['count']
    full_name = d['node'] ['owner'] ['full_name']
    img_class= ""
    if(('accessibility_caption' in d['node']) and ('accessibility_caption' in d['node'] != None)):
        img_class = d ['node'] ['accessibility_caption']
    age = d['node'] ['owner'] ['highlight_reel_count']
    id_user = d['node'] ['owner'] ['id']
    username = d['node'] ['owner'] ['username']
    date_post = d['node'] ['taken_at_timestamp']
    date_p = datetime.fromtimestamp(date_post)
    if('edge_media_to_parent_comment' in d['node']):
        edge_media_to_parent_comment = d['node'] ['edge_media_to_parent_comment'] ['edges']
        for f in edge_media_to_parent_comment:
            date_comment = f['node'] ['created_at']
            date_c = datetime.fromtimestamp(date_comment)
            user_comment = f['node'] ['owner'] ['username']
            text_comment = f['node'] ['text']
            for has in text_comment:
                hashtag=re.findall(r'\#\w+', text_comment)
                hashtags = hashtag + hashtag1
                nr_hashtags = len(hashtags)
            newl.append({'caption': caption, 'id_post':id_post, 'nr_comments':nr_comments, 'date_c':date_c, 
                         'nr_likes':nr_likes,' text_comment': text_comment, 'location_name':location_name, 'lat':lat, 
                         'lng':lng,'img_class':img_class,
                         'followers':followers, 'followed':followed, 'full_name':full_name, 'age':age,
                         'id_user':id_user, 'username':username, 'user_comment':user_comment, 'date_p':date_p, 
                         'hashtags':hashtags, 'nr_hashtags':nr_hashtags})
            

In [8]:
ds_newl = pd.DataFrame(newl)
ds_newl.to_csv(r'C:\Users\valen\Downloads\instagram_posts.csv')

In [ ]:
Image_list = []
import time
from clarifai.rest import ClarifaiApp

clarifai_key = "yourKey"
app = ClarifaiApp(api_key=clarifai_key)
model = app.public_models.general_model

for filename in glob.glob(r'C:\Users\valen\instagram_posts\photos\*.jpg'):
    time.sleep(5)
    print(filename)
    response = model.predict_by_filename(filename) 
    if(response['status']['description'] == "Ok"):
        for concept in response["outputs"][0]["data"]["concepts"]:
            name = concept["name"]
            value = concept["value"]
            Image_list.append({'immagine': filename, 'name': name,
        'value': value})
    
print(Image_list)

In [29]:
len(Image_list)

6540

In [30]:
import pandas as pd
ds_img = pd.DataFrame(Image_list)

len(ds_img)

6540

In [31]:
ds_img.to_csv(r'C:\Users\valen\Downloads\instagram_images.csv')

In [342]:
import pandas as pd 
full=pd.read_csv(r'C:\Users\valen\Downloads\instagram_posts.2.csv', sep=',', na_values='NaN')

In [ ]:
unique_hash_post=full['hashtags'].unique()
print(unique_hash_post)

In [ ]:
hashtag_total= unique_hash_post.tolist()
print(hashtag_total)


In [ ]:
newlist = [word for line in hashtag_total for word in line.split()]
print (newlist)

In [470]:
newlist = [w.replace("[", "''").replace(",","''").replace("]","''").replace("'"," ")for w in newlist]

In [471]:
ds_newlist = pd.DataFrame(newlist)


In [472]:
ds_newlist.head()

,0
0,#trekking
1,#escursionismo
2,#montagna
3,#trentino
4,#molveno


In [473]:
len(ds_newlist)

4275

In [474]:
ds_newlist.to_csv(r'C:\Users\valen\Downloads\instagram_hashtags.csv')